In [1]:
# LSTM for international airline passengers problem with window regression framing
import numpy
import numpy as np
import keras
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from keras.utils.vis_utils import plot_model

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [3]:
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset
# dataframe = read_csv('w_d_v.csv', usecols=[7], engine='python', skipfooter=3)
dataframe = read_csv('00.txt', usecols=[0], engine='python',skiprows=-1)
all_data = read_csv('all_data.csv', usecols=[7], engine='python', skipfooter=3)
dataset = dataframe.values
allData=all_data.values

In [4]:
look_back = 3
trainX, trainY = create_dataset(dataset, look_back)
AllX, AllY = create_dataset(allData, look_back)
trainY=numpy.reshape(trainY,(trainY.shape[0],-1))
AllY=numpy.reshape(AllY,(AllY.shape[0],-1))

In [5]:
encX = OneHotEncoder()
encX.fit(AllX)
encY = OneHotEncoder()
encY.fit(AllY)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [6]:
trainX_one=encX.transform(trainX).toarray()
train_X=numpy.reshape(trainX_one,(trainX_one.shape[0],look_back,-1))
train_Y=encY.transform(trainY).toarray()

In [7]:
a_train, a_test, b_train, b_test = train_test_split(train_X, train_Y, test_size=0.1, random_state=42)

In [9]:
model = Sequential()
# model.add(Embedding(max_features, output_dim=256))
model.add(LSTM(512,return_sequences=True,
               input_shape=(3, a_train.shape[2]),name="lstm"))  # returns a sequence of vectors of dimension 32
model.add(LSTM(300))  # return a single vector of dimension 32
model.add(Dropout(0.2))
model.add(Dense(250,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(a_train.shape[2],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [10]:
plot_model(model, to_file='lstm.png')

In [11]:
#batchsize批尺寸
model.fit(a_train, b_train, epochs=100, batch_size=8, verbose=2, validation_data=(a_test, b_test))

Train on 226 samples, validate on 26 samples
Epoch 1/100
 - 3s - loss: 4.2319 - acc: 0.2212 - val_loss: 2.8721 - val_acc: 0.3846
Epoch 2/100
 - 2s - loss: 2.7530 - acc: 0.2743 - val_loss: 2.6926 - val_acc: 0.3846
Epoch 3/100
 - 2s - loss: 2.5303 - acc: 0.3097 - val_loss: 2.6912 - val_acc: 0.3846
Epoch 4/100
 - 2s - loss: 2.4803 - acc: 0.3186 - val_loss: 2.7821 - val_acc: 0.3846
Epoch 5/100
 - 2s - loss: 2.4673 - acc: 0.3407 - val_loss: 2.8004 - val_acc: 0.3846
Epoch 6/100
 - 2s - loss: 2.4003 - acc: 0.3053 - val_loss: 2.8364 - val_acc: 0.3846
Epoch 7/100
 - 2s - loss: 2.4104 - acc: 0.3230 - val_loss: 2.8217 - val_acc: 0.3846
Epoch 8/100
 - 2s - loss: 2.3647 - acc: 0.3451 - val_loss: 2.8897 - val_acc: 0.4231
Epoch 9/100
 - 2s - loss: 2.3150 - acc: 0.3274 - val_loss: 3.1323 - val_acc: 0.3462
Epoch 10/100
 - 2s - loss: 2.3767 - acc: 0.3053 - val_loss: 2.8632 - val_acc: 0.4615
Epoch 11/100
 - 2s - loss: 2.2500 - acc: 0.3274 - val_loss: 3.0226 - val_acc: 0.3462
Epoch 12/100
 - 2s - loss: 2.

Epoch 97/100
 - 2s - loss: 0.5647 - acc: 0.7611 - val_loss: 5.3307 - val_acc: 0.1538
Epoch 98/100
 - 2s - loss: 0.5510 - acc: 0.7611 - val_loss: 5.4140 - val_acc: 0.2308
Epoch 99/100
 - 2s - loss: 0.5543 - acc: 0.7788 - val_loss: 5.3167 - val_acc: 0.2308
Epoch 100/100
 - 2s - loss: 0.5593 - acc: 0.7832 - val_loss: 5.0856 - val_acc: 0.2308


In [12]:
model.evaluate(train_X, train_Y, batch_size=64, verbose=2, sample_weight=None)

[0.9720331950793191, 0.7341269812886677]

In [13]:
trainPredict = model.predict(train_X)
D=np.argmax(train_Y,axis = 1)
E=np.argmax(trainPredict,axis = 1)

In [16]:
print(D)
print(E)

[ 31  31  31 130  65   1  26   1 131   1  26 108  52 131  26  26 181   1
   1 150 150 150 150 126  64   1   1 181 150 181 150 150 181 150 150 181
   1 108 181 150 157 150 150 181 150 167   1 181 150 181 181 150 150 150
   1 181 181 181 157 181 181 181   1 181 150 181 181 181 181   1 181 150
 181 150 150 150 150 196 150 150 150 139  68 150 181   1   1 108  15  92
 150 116 196 108 193  65  90  64 159 159 181 181 181   1 197 159  28 159
 108 181 150 181 108 181 181 181 150   1 108  97 131 108 159  28  28 146
 108   1 150 181 150 150   1 181 162 181   1 196  34  52  70 150  84  51
 196  79 181 181 150 181   1 181   1 181   1 181 150 181 181 181 181 150
 150 181 181 181 181 181 181 181 181 116  32 181 181 150   1 197 197  52
 119 198 181 181  52 104   1 150 181 181 181   7 150 181 181   1   1 181
 181 181 181 108  57  57 191  15 162 132 132  56 181 181   1 198 181 181
   1 181 181 150 150 181 181 150 150 181   1 181 157   1 181   1 181 181
 116 116 150 150 181 181 181 181 184 108 157 157   

In [19]:
A=0 #total number of right
for i,t in enumerate(E):
    if D[i]==t :
        A=A+1
print(A/D.shape[0])

0.7341269841269841
